In [5]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
import dotenv
import os
from sqlalchemy import create_engine
import urllib
from langchain.prompts.chat import ChatPromptTemplate
from lib.etl import DatabaseETL
etl = DatabaseETL('../parameters/config.json')

dotenv.load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
server = os.getenv('AzureServer')
database = os.getenv('AzureDatabase')    
username = os.getenv('AzureUsername')
password = os.getenv('AzurePassword')
driver = os.getenv('AzureDriver')

# Create the connection URL
params = urllib.parse.quote_plus(
    'DRIVER={' + driver + '};'
    'SERVER=' + server + ';'
    'DATABASE=' + database + ';'
    'UID=' + username + ';'
    'PWD=' + password
)
# Create the engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params))
# Create an instance of SQLDatabase
db = SQLDatabase(engine)
# db = etl.create_engine()


# Set up the Agent template ( Custom Instructions)
instructions = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a helpful AI assistant expert in querying Microsoft SQL Server databases. Remember to use SQL Server syntax, such as using 'TOP' instead of 'LIMIT' for restricting the number of rows returned in a query. Your task is to construct SQL queries to find answers to user's questions about Customers, Parts, and Orders, following SQL Server query conventions.
         Use following context to create the SQL query. Context:
         LineItems table contains information about parts that are purchased with each order. lineItems table including LineItemId as primary key, PartCost, PartNumber, Date and OrderId which is used to identify the order in which the part was purchased.
         
         Orders table contains information about orders placed by customers including OrderNumber, Revenue, Date, OrderId as primary key, and CustomerId which is primary key frp, Customers table used to identify the customer who placed the order.
         
         Customers table contains information about Customers who have purchased the parts and their order. Customers table including CustomerId as primary and is used in Orders table as foreign key.
        """
         ),
        ("user", "{question}\n ai: "),
    ]
)

# Create an instance of an Agent
sql_agent = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,)
# Run the agent
sql_agent.run(instructions.format(
        question="Which customer spend the most amount of money based on the parts cost?"
  ))



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Customers, LineItems, Orders
Thought: I should query the schema of the Customers, LineItems, and Orders tables to see what columns I can use in my query.
Action: sql_db_schema
Action Input: Customers, LineItems, Orders
Observation: 
CREATE TABLE [Customers] (
	[CustomerId] BIGINT NOT NULL, 
	[CustomerName] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	CONSTRAINT [PK_Customers] PRIMARY KEY ([CustomerId])
)

/*
3 rows from Customers table:
CustomerId	CustomerName
1	Customer_118
2	Customer_405
3	Customer_100
*/


CREATE TABLE [LineItems] (
	[OrderId] BIGINT NOT NULL, 
	[LineItemId] BIGINT NOT NULL, 
	[PartCost] FLOAT(53) NULL, 
	[PartNumber] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Date] DATETIME NULL, 
	CONSTRAINT [FK_LineItems_Orders] FOREIGN KEY([OrderId]) REFERENCES [Orders] ([OrderId])
)

/*
3 rows from LineItems table:
OrderId	LineItemId	PartCost	PartNumber	Da

'Customer_65'